In [14]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [15]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [16]:
investor = Investor(
                app_id="38cFFxHReqfzLBfz",                                 
                app_secret="XkyRuCPuh4UOOg0hHmIZfSq8vS+jqGo5zIfEu2Wgarw=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="babi-D")            

account_info = deri.get_account_info()

account_info       

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [17]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
normalized=True,)

In [18]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

lastSequence        time   open   high    low  close    volume  value
0               0  1694365200  71.00  71.50  70.50  71.25  17444234      0
1               0  1694451600  71.50  71.75  71.25  71.25   7322782      0
2               0  1694538000  71.50  71.50  71.00  71.25  13280430      0
3               0  1694624400  71.25  71.50  70.75  71.50   9560217      0
4               0  1694710800  71.75  71.75  71.00  71.25  20750854      0
..            ...         ...    ...    ...    ...    ...       ...    ...
495             0  1758646800  39.00  40.00  39.00  39.75  17328719      0
496             0  1758733200  39.75  40.00  39.25  40.00  18379790      0
497             0  1758819600  39.75  39.75  39.00  39.25  14121257      0
498             0  1759078800  39.75  40.00  39.25  39.75  14664521      0
499             0  1759165200  40.00  40.75  39.75  40.50  37282120      0

[500 rows x 8 columns]

In [19]:
df.head(10)

lastSequence        time   open   high    low  close    volume  value
0             0  1694365200  71.00  71.50  70.50  71.25  17444234      0
1             0  1694451600  71.50  71.75  71.25  71.25   7322782      0
2             0  1694538000  71.50  71.50  71.00  71.25  13280430      0
3             0  1694624400  71.25  71.50  70.75  71.50   9560217      0
4             0  1694710800  71.75  71.75  71.00  71.25  20750854      0
5             0  1694970000  71.25  71.25  70.50  70.75  15823767      0
6             0  1695056400  71.00  71.25  70.25  70.25  13997136      0
7             0  1695142800  70.50  70.50  69.50  69.75  33323529      0
8             0  1695229200  69.50  70.50  69.25  70.00  17096646      0
9             0  1695315600  70.00  70.75  69.75  70.25  17755423      0

In [20]:
session.set_keyspace('data_stock')

In [22]:
session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick1_1068_data (
        symbol text,
        time timestamp,
        open_price float,
        high_price float,
        low_price float,
        close_price float,
        volume bigint,
        value float,
        PRIMARY KEY (symbol, time)
    ) WITH CLUSTERING ORDER BY (time ASC);
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

✅ Keyspace และ Table พร้อมใช้งาน!


In [25]:
# ✅ โหลดรายชื่อหุ้นทั้งหมด
try:
    symbols_df = pd.read_excel("C:/Users/Admin/OneDrive/Desktop/Stock-quadrant-quantitative-analysis-using-deep-learning-classification/1.Final_file/1.Pull_the_name_of_Thai_stocks/stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'stocth_names.xlsx' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ ฟังก์ชันดึง candlestick จาก API
def get_candlestick_data(symbol):
    try:
        res = market.get_candlestick(  # ✅ ต้องกำหนด market ก่อนใช้
            symbol=symbol,
            interval="1d",
            limit=1000,
            normalized=True
        )
        return res
    except Exception as e:
        print(f"❌ ไม่สามารถดึงข้อมูลของ {symbol}: {e}")
        return None

# ✅ ฟังก์ชัน insert ข้อมูลลง Cassandra
def insert_financial_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        # ✅ แปลง timestamp ให้ถูกต้อง
        timestamp = datetime.fromtimestamp(res["time"][i])

        session.execute("""
            INSERT INTO candlestick1_1068_data (
                symbol, time, open_price, high_price, low_price,
                close_price, volume, value
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            timestamp,
            res["open"][i],
            res["high"][i],
            res["low"][i],
            res["close"][i],
            res["volume"][i],
            res["value"][i]
        ))

    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# ✅ ดึงและบันทึกข้อมูลทุกหุ้น
for symbol in symbols:
    print(f"\n📦 กำลังโหลดข้อมูล {symbol} ...")
    res = get_candlestick_data(symbol)
    insert_financial_data(symbol, res)
    time.sleep(2)  # 🕒 ลดโอกาสถูก block จาก API

print("\n🎉 เสร็จสิ้นการโหลดข้อมูลทั้งหมด!")


📦 กำลังโหลดข้อมูล 24CS ...
✅ เพิ่มข้อมูล 729 รายการของหุ้น 24CS สำเร็จ!

📦 กำลังโหลดข้อมูล 2S ...
✅ เพิ่มข้อมูล 728 รายการของหุ้น 2S สำเร็จ!

📦 กำลังโหลดข้อมูล 3BBIF ...
✅ เพิ่มข้อมูล 728 รายการของหุ้น 3BBIF สำเร็จ!

📦 กำลังโหลดข้อมูล A ...
✅ เพิ่มข้อมูล 700 รายการของหุ้น A สำเร็จ!

📦 กำลังโหลดข้อมูล A5 ...
✅ เพิ่มข้อมูล 729 รายการของหุ้น A5 สำเร็จ!

📦 กำลังโหลดข้อมูล AAI ...
✅ เพิ่มข้อมูล 710 รายการของหุ้น AAI สำเร็จ!

📦 กำลังโหลดข้อมูล AAV ...
✅ เพิ่มข้อมูล 728 รายการของหุ้น AAV สำเร็จ!

📦 กำลังโหลดข้อมูล ABM ...
✅ เพิ่มข้อมูล 727 รายการของหุ้น ABM สำเร็จ!

📦 กำลังโหลดข้อมูล ACAP ...
✅ เพิ่มข้อมูล 118 รายการของหุ้น ACAP สำเร็จ!

📦 กำลังโหลดข้อมูล ACC ...
✅ เพิ่มข้อมูล 727 รายการของหุ้น ACC สำเร็จ!

📦 กำลังโหลดข้อมูล ACE ...
✅ เพิ่มข้อมูล 728 รายการของหุ้น ACE สำเร็จ!

📦 กำลังโหลดข้อมูล ACG ...
✅ เพิ่มข้อมูล 723 รายการของหุ้น ACG สำเร็จ!

📦 กำลังโหลดข้อมูล ADB ...
✅ เพิ่มข้อมูล 728 รายการของหุ้น ADB สำเร็จ!

📦 กำลังโหลดข้อมูล ADD ...
✅ เพิ่มข้อมูล 728 รายการของหุ้น ADD สำเร็จ!

📦 กำล

In [ ]:
name_stock = "A5"
rows = session.execute(f"SELECT * FROM candlestick1_1068_data WHERE symbol = '{name_stock}'")


In [ ]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)
df.head()


symbol       time  open_price  high_price  low_price  close_price  \
0     A5 2022-03-29        1.97        2.10       1.97         2.06   
1     A5 2022-03-30        1.99        2.02       1.96         1.99   
2     A5 2022-03-31        1.98        2.06       1.98         2.00   
3     A5 2022-04-01        2.12        2.22       2.02         2.06   
4     A5 2022-04-04        2.74        2.74       2.18         2.20   

         volume      value  
0  5.789567e+07   28666337  
1  6.399037e+07   32160278  
2  4.048799e+07   20186147  
3  6.379672e+08  315307817  
4  1.922967e+09  762254699

In [ ]:
import plotly.graph_objects as go

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    name=name_stock
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    name='Volume'
))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    title=f'📈 {name_stock} - Candlestick Chart with Volume',
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()
